 <h1 style="text-align: center; color: RED">ÖLÇÜMLEME PROBLEMLERİ</h1>

 <h2 style="text-align: center; color: RED">SORTİNG REVİEWS(YORUM SIRALAMA)</h2>

Yorum sıralama (Sorting Reviews), kullanıcıların yaptığı incelemeleri belirli bir kritere göre önceliklendirerek en önemli veya en güvenilir yorumları en üstte göstermeyi amaçlar.Bir e-ticaret platformunda, film veya ürün değerlendirme sistemlerinde en iyi sıralama stratejisi, kullanıcıların en faydalı yorumları görmesini sağlar.

In [45]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import math
import scipy.stats as st
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
pd.set_option('display.float_format', lambda x : '% .5f' % x)

<span style="color: blue; font-weight: bold;">1-UP-DOWN DIFFERENT SCORE</span>

(up_ratings)-(down_ratings)

Yorum 1 : 600 like 400 dislike

Yorum 2 : 5500 like 4500 dislike

Bu yorumlardan hangisini şecerdiniz?

In [6]:
def score_up_down_diff(up, down):
    return up-down

In [8]:
score_up_down_diff(600, 400)#yorum1

200

In [10]:
score_up_down_diff(5500, 4500)#yorum2

1000

* Görünüşe göre 2. yorum seçilir. Fakat şöyle bir sorun var biri 1000 yorum almışken diğeri 10000 yorum almış.Yüzdelik skorlara bakıldığında yorum1'in pozitif skoru %60 iken yorum2'nin pozitif skoru %55 görünüyor. Yani up-down bizi istediğimiz sonuca götürmeyecektir.

<span style="color: blue; font-weight: bold;">2-AVERAGE RATİNG(ORTALAMA PUANI)</span>

Score = Average Rating = (up ratings) / (all ratings)

In [23]:
def score_average_rating(up,down):
    if up + down ==0:
        return 0
    return up / (up + down)

In [25]:
score_average_rating(600, 400)#yorum1

0.6

In [27]:
score_average_rating(5500, 4500)#yorum2

0.55

Başka bir seneryo ele alalım

2 like 0 dislike toplamda 2 yorum

100 like 1 dislike toplamda 101 yorum alan 2 ayrı yorum olsun bunları avera_rating_score göre secelim.

In [30]:
score_average_rating(2, 0)#yorum1

1.0

In [32]:
score_average_rating(100, 1)#yorum2

0.9900990099009901

* Yukarıdaki sonuçlara bakıldığında 2 beğeni alan yorum ilk sırada yer alır. Fakat görüyoruz ki 2. yorumun beğenisi daha fazla.
Yani bu yöntemde bizi istediğimiz sonuca ulaştırmadı.Oran bilgisini yakaladık fakat frekans bilgisi göz ardı edilmiş oldu.

<span style="color: blue; font-weight: bold;">3-WİLSON LOWER BOUND SCORE</span>

Wilson Lower Bound Score (WLB), derecelendirme sistemlerinde (örneğin ürün yorumları, kullanıcı puanları gibi) bir içeriğin güvenilirliğini ölçmek için kullanılan istatistiksel bir yöntemdir.Bir ürünün aldığı pozitif ve negatif oyları dikkate alarak, belirli bir güven aralığında (genelde %95) "bu ürünün gerçek puanı en az ne olabilir?" sorusuna yanıt verir.Wilson Lower Bound, sadece “puanı yüksek” değil, “puanı yüksek ve yeterince oy almış” içerikleri öne çıkarmak için kullanılır.



In [47]:
def wilson_lower_bound(up, down, confidence=0.95):
    """
    Wilson Lower Bound Score hesapla

    - Bernoulli parametresi p için hesaplanacak güven aralığının alt sınırı WLB skoru olarak kabul edilir.
    - Bernoulli dağılımı, iki olasılıklı (binary) olayları modelleyen bir istatistiksel dağılımdır
    - Başarı (1) → Örneğin, "Beğeni aldı" (Upvote)
    - Başarısızlık (0) → Örneğin, "Beğeni almadı" (Downvote)
    - Hesaplanacak skor ürün sıralaması için kullanılır.
    - Not:
    Eğer skorlar 1-5 arasıdaysa 1-3 negatif, 4-5 pozitif olarak işaretlenir ve bernoulli'ye uygun hale getirilebilir.
    Bu beraberinde bazı problemleri de getirir. Bu sebeple bayesian average rating yapmak gerekir.
    Bernoulli dağılımı, sadece iki olası çıktısı olan olayları modeller:
    Başarı (1)
    Başarısızlık (0)

    Parameters
    ----------
    up: int
        up count
    down: int
        down count
    confidence: float
        confidence

    Returns
    -------
    wilson score: float

    """
    n = up + down
    if n == 0:
        return 0
    z = st.norm.ppf(1 - (1 - confidence) / 2)
    phat = 1.0 * up / n
    return (phat + z * z / (2 * n) - z * math.sqrt((phat * (1 - phat) + z * z / (4 * n)) / n)) / (1 + z * z / n)

In [49]:
wilson_lower_bound(600,400)

0.5693094295142663

In [51]:
wilson_lower_bound(5500,4500)

0.5402319557715324

In [53]:
wilson_lower_bound(2,0)

0.3423802275066531

In [55]:
wilson_lower_bound(100,1)

0.9460328420055449

<span style="color: blue; font-weight: bold;">ÖRNEK UYGULAMA</span>

In [59]:
up = [15, 70, 14, 4, 2, 5, 8, 37, 21, 52, 28, 147, 61, 30, 23, 40, 37, 61, 54, 18, 12, 68]
down = [0, 2, 2, 2, 15, 2, 6, 5, 23, 8, 12, 2, 1, 1, 5, 1, 2, 6, 2, 0, 2, 2]
comments = pd.DataFrame({"up": up, "down": down})

In [63]:
# score_pos_neg_diff
comments["score_pos_neg_diff"] = comments.apply(lambda x: score_up_down_diff(x["up"],
                                                                             x["down"]), axis=1)

In [65]:
# score_average_rating
comments["score_average_rating"] = comments.apply(lambda x: score_average_rating(x["up"], x["down"]), axis=1)

In [67]:
# wilson_lower_bound
comments["wilson_lower_bound"] = comments.apply(lambda x: wilson_lower_bound(x["up"], x["down"]), axis=1)

In [69]:
comments

,up,down,score_pos_neg_diff,score_average_rating,wilson_lower_bound
0,15,0,15,1.00000,0.79612
1,70,2,68,0.97222,0.90426
2,14,2,12,0.87500,0.63977
3,4,2,2,0.66667,0.29999
4,2,15,-13,0.11765,0.03288
5,5,2,3,0.71429,0.35893
6,8,6,2,0.57143,0.32591
7,37,5,32,0.88095,0.75000
8,21,23,-2,0.47727,0.33755
9,52,8,44,0.86667,0.75835


In [71]:
comments.sort_values('wilson_lower_bound', ascending=False)

,up,down,score_pos_neg_diff,score_average_rating,wilson_lower_bound
11,147,2,145,0.98658,0.95238
12,61,1,60,0.98387,0.91413
1,70,2,68,0.97222,0.90426
21,68,2,66,0.97143,0.90168
18,54,2,52,0.96429,0.87881
15,40,1,39,0.97561,0.87405
13,30,1,29,0.96774,0.83806
16,37,2,35,0.94872,0.83114
19,18,0,18,1.00000,0.82412
17,61,6,55,0.91045,0.81807
